# V - Phylogenetic Tree Construction

With this code you can construct a phylogenetic tree. You only need a list of Genbank accession numbers.

You can find all material in my GitHub repository

## Set your working directory

First, set up your working directory. Standard, this is the location where this jupyter notebook is situated. 

In [ ]:
import os
wdir = os.getcwd()
print(wdir)

## Obtain sequences

We start from a file with accession numbers and the country of origin. 

I got the accession numbers from https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus?SeqType_s=Nucleotide&VirusLineage_ss=Severe%20acute%20respiratory%20syndrome%20coronavirus%202,%20taxid:2697049

There you can choose your countries of interest. I chose 5 that will create a phylogenetic with thirty sequences in total. 

In [ ]:
import csv
from Bio import SeqIO
from Bio import Entrez

fasta_file = {}
ct = 0

Entrez.email = "A.N.Other@example.com"
with open('accessions.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        print(row)
        with Entrez.efetch(db="nucleotide", rettype="gb", retmode = "text", id = row[0]) as handle:
            for seq_record in SeqIO.parse(handle, "gb"):
                fasta_file[row[1]] = str(seq_record.seq)
                ct += 1  
      
print("Number of sequences in file:", ct)

# save all sequences in the multi-fasta file "sequences_general.txt"

f = open("sequences.txt","w+")
for keys,values in fasta_file.items():
    f.write(">" + str(keys) + "\n" + str(values) + "\n")
f.close()

## Multiple sequence alignment (MSA)

You can choose between the locally installed or browser-based Clustal Omega. In this tutorial, we use the locally installed version. You can find the Clustal Omega file on github. You only have to unzip clustalo.zip and store it in the same folder as this notebook. 

In [ ]:
from Bio.Align.Applications import ClustalOmegaCommandline

in_file = "sequences.txt"

out_file = "alignment_covid.fasta"

clustalomega_cline = ClustalOmegaCommandline(infile = in_file, outfile = out_file, verbose=True, auto=False)

print(clustalomega_cline)

"""
# WINDOWS
# Python will fill out clustal omega in the command line for you!
path = "r'" + str(wdir) #.'
input_command = '"' + wdir + '/clustal-omega-1.2.2-win64/' + str(clustalomega_cline)[0:8] + '" ' + str(clustalomega_cline)[9:] + '-threads=1'
os.system(input_command)

"""
# MAC
# Python will fill out clustal omega in the command line for you!
input_command = "./" + str(clustalomega_cline)
os.system(input_command)


You can find the result of the msa in your working directory. It will be a fasta-file with the same name as the 'out_file'.

## Draw tree

Before drawing the actual tree, we create a distance matrix. This forms the basis for the tree. 

In [ ]:
out_file = "alignment_covid.fasta"

# Read alignment file
from Bio import AlignIO
align = AlignIO.read(out_file, "fasta")
print(align)

# Calculate distance matrix
from Bio.Phylo.TreeConstruction import DistanceCalculator
calculator = DistanceCalculator('identity')
dm = calculator.get_distance(align)
print(dm)

# Drawing the phylogenetic tree using UPGMA (unweighted pair group method with arithmetic mean) method

In [ ]:
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
constructor = DistanceTreeConstructor()
tree = constructor.upgma(dm)

import matplotlib
import matplotlib.pyplot as plt

def plot_tree(tree, output_file):
    # set the size of the figure
    fig = plt.figure(figsize=(50, 40), dpi=100)
    # fig.set_size_inches(10, 20) # in inches
    plt.rcParams.update({'font.size':30})
    axes = fig.add_subplot(1, 1, 1)
    fig1 = plt.gcf()
    Phylo.draw(tree, axes=axes, branch_labels=None)
    Phylo.draw_ascii(tree)
    fig1.savefig(output_file, dpi=100)
    return

plot_tree(tree,"tree.jpg")